# Final Project 

### Alex Tarasov, Eric Holguin, Gordon Zhong

This is a toxic comment classifier using Naive-Bayes 

In [1]:
import numpy as np
import pandas as pd 
import re
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn
from heapq import nlargest
from textblob.classifiers import NaiveBayesClassifier
import warnings
warnings.filterwarnings("ignore")

#### Create list of labels

In [53]:
labelName = []
toxic = 0
labelName.append("Toxic")
severe_toxic = 1
labelName.append("Severe toxic")
obscene = 2
labelName.append("Obscene")
threat = 3
labelName.append("Threat")
insult = 4
labelName.append("Insult")
identity_hate = 5
labelName.append("Identity hate")

In [54]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Read in the dataset obtained from Kaggle. 

Data is already split into training and testing

In [55]:
train = pd.read_csv('./all/train.csv')
test = pd.read_csv('./all/test.csv')
test_labels = pd.read_csv('./all/test_labels.csv')
test = pd.merge(test, test_labels, on='id', how='inner')

#### Clean the data

Removing punctuation, duplicates and stopwords

In [56]:
stripNonAlpha = re.compile("[^a-zA-Z]")
stripDuplicates = re.compile(r'(.)\1+')

stopwords = nltk.corpus.stopwords.words("english")
stopwords = " ".join(stopwords).replace("'","")
stopwords = stripDuplicates.sub(r'\1', stopwords).split()
stopwords = set(stopwords)

In [57]:
def conditionText(text):
    text = text.lower()
    text = text.replace("'","")
    text = text.replace("-","")
    text = text.replace("_","")
    text = stripNonAlpha.sub(' ', text)
    text = stripDuplicates.sub(r'\1', text)
    words = text.split()
    words = [word for word in words if word not in stopwords]
    return " ".join(words)

In [58]:
print("Building training arrays ...")
train_X = [row.comment_text for row in train.itertuples() if row.toxic >=0]
train_y = [None]*6
train_y[toxic] = [row.toxic != 0 for row in train.itertuples() if row.toxic >=0]
train_y[severe_toxic] = [row.severe_toxic != 0 for row in train.itertuples() if row.toxic >=0]
train_y[obscene] = [row.obscene != 0 for row in train.itertuples() if row.toxic >=0]
train_y[threat] = [row.threat != 0 for row in train.itertuples() if row.toxic >=0]
train_y[insult] = [row.insult != 0 for row in train.itertuples() if row.toxic >=0]
train_y[identity_hate] = [row.identity_hate != 0 for row in train.itertuples() if row.toxic >=0]
train = None
print("Done building training arrays!")

print("Vectorizing training data ...")
cv = CountVectorizer(preprocessor=conditionText)
train_X = cv.fit_transform(train_X)
freqTransform = TfidfTransformer(use_idf=False).fit(train_X)
train_X = freqTransform.transform(train_X)
print("Done vectorizing training data!")

print("Training ...")
gnb = []
for i in range(6):
    gnb.append(MultinomialNB())
    gnb[i].fit(train_X, train_y[i])

print("Done training!")

Building training arrays ...
Done building training arrays!
Vectorizing training data ...


TypeError: 'zip' object is not subscriptable

In [18]:
print("Building testing arrays ...")
test_X = [row.comment_text for row in test.itertuples() if row.toxic >=0]
test_y = [None]*6
test_y[toxic] = [row.toxic != 0 for row in test.itertuples() if row.toxic >=0]
test_y[severe_toxic] = [row.severe_toxic != 0 for row in test.itertuples() if row.toxic >=0]
test_y[obscene] = [row.obscene != 0 for row in test.itertuples() if row.toxic >=0]
test_y[threat] = [row.threat != 0 for row in test.itertuples() if row.toxic >=0]
test_y[insult] = [row.insult != 0 for row in test.itertuples() if row.toxic >=0]
test_y[identity_hate] = [row.identity_hate != 0 for row in test.itertuples() if row.toxic >=0]
test = None
print("Done building testing arrays!")

print("Vectorizing testing data ...")
test_X = cv.transform(test_X)
test_X = freqTransform.transform(test_X)
print("Done testing data!")

Building testing arrays ...
Done building testing arrays!
Vectorizing testing data ...
Done testing data!


In [19]:
print("Clasifying ...")
test_z = []
for g in gnb:
    test_z.append(g.predict(test_X))
print("Done clasifying data!")
print("\n")

for z, y, l in zip(test_z, test_y, labelName):
    print(l+": ")
    print("Accuracy percent:", sklearn.metrics.accuracy_score(y, z)*100,"\n")
    print(sklearn.metrics.classification_report(y, z))

Clasifying ...
Done clasifying data!


Toxic: 
Accuracy percent: 92.2895370283535 

             precision    recall  f1-score   support

      False       0.92      1.00      0.96     57888
       True       0.92      0.21      0.34      6090

avg / total       0.92      0.92      0.90     63978

Severe toxic: 
Accuracy percent: 99.42636531307637 

             precision    recall  f1-score   support

      False       0.99      1.00      1.00     63611
       True       0.00      0.00      0.00       367

avg / total       0.99      0.99      0.99     63978

Obscene: 
Accuracy percent: 94.95451561474256 

             precision    recall  f1-score   support

      False       0.95      1.00      0.97     60287
       True       0.98      0.13      0.23      3691

avg / total       0.95      0.95      0.93     63978

Threat: 
Accuracy percent: 99.67019913095126 

             precision    recall  f1-score   support

      False       1.00      1.00      1.00     63767
       True     